In [1]:
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
import numpy as np
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os # interface with underlying OS that python is running on
import sys

In [2]:
# LOAD IN FILE
#main_dir = 'D:/Makaleler/makale19 speech recog/ravdes/ravdes/Audio_Speech_Actors_01-24/'
main_dir = "C:/Laptop3/Makaleler/makale19 speech recog/emodb/wav/"
#sub_dir = os.listdir(main_dir)
x, sr = librosa.load(main_dir+'/03a05Wa.wav')
# PLAY any AUDIO FILE
#librosa.output.write_wav('MaleNeutral.wav', x, sr)
Audio(data=x, rate=sr)

In [3]:
"""
def extract_feature(file_name, offst=0.5):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast',offset=offst)
    
    stft = np.abs(librosa.stft(X))
    chroma_cq = librosa.feature.chroma_cqt(y=X, sr=sample_rate)
    cqt=np.mean(chroma_cq,axis=1)
    
    #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate,n_fft=2048).T,axis=0)
    #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate,n_mels=128,fmax=8000).T,axis=0)
    #mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,cqt,mel,contrast,tonnetz
"""
def extract_feature(file_name, offst=0.5):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast',offset=offst)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate,n_fft=2048).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate,n_mels=128,fmax=8000).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def extract_featurev2(file_name):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast', sr=None)   
    #mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=64).T, axis=0)
    C = np.abs(librosa.cqt(X, sr=sr, fmin=librosa.note_to_hz('C2'), n_bins=60))
    #d=librosa.amplitude_to_db(C)
    d=librosa.power_to_db(C)
    cqt = np.mean(d,axis=1)
    return cqt

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [6]:
a= []
b= []
c= []
d= []
e= []
#from warnings import warn
import warnings
# run block of code and catch warnings
with warnings.catch_warnings():
	# ignore all caught warnings
	warnings.filterwarnings("ignore")
	# execute code that will generate warnings

X4= pd.DataFrame()
cc=0
for root, dirs, files in os.walk(main_dir, topdown=False):
    for f in files:
        #print(os.path.join(root, f))
        #print(f)
        a,b,c,d,e=extract_feature(os.path.join(root, f))
        #a=extract_feature(main_dir + i + '/' + f)
        tot= []
        for x in a:tot.append(x)
        for x in b:tot.append(x)
        for x in c:tot.append(x)
        for x in d:tot.append(x)
        for x in e:tot.append(x)
        
        #X4[cc]=pd.DataFrame([tot])
        X4[cc]=np.asarray(tot)
        cc+=1;
        #print(tot)

In [7]:
len(a),len(b),len(c),len(d),len(e),

(40, 12, 128, 7, 6)

In [8]:
X6= pd.DataFrame(X4.T)

In [9]:
type(X6), X6.shape

(pandas.core.frame.DataFrame, (535, 193))

In [10]:
def decompose_emodb():
        #EMODB_PATH = '/kaggle/input/berlin-database-of-emotional-speech-emodb/wav'
        emotion = []
        emov2 = []
        path = []
        gender = []  
        
        for root, dirs, files in os.walk(main_dir):
            for name in files:
                #print(name)
                if name[0:2] in ['03','10','11','12','15']: #'0310111215':  # MALE
                    if name[5] == 'W':  # Ärger (Wut) -> Angry
                        emotion.append('male_angry')
                        emov2.append(1)
                    elif name[5] == 'L':  # Langeweile -> Boredom
                        emotion.append('male_bored')
                        emov2.append(2)
                    elif name[5] == 'E':  # Ekel -> Disgusted
                        emotion.append('male_disgust')
                        emov2.append(4)
                    elif name[5] == 'A':  # Angst -> Angry
                        emotion.append('male_fear')
                        emov2.append(5)
                    elif name[5] == 'F':  # Freude -> Happiness
                        emotion.append('male_happy')
                        emov2.append(6)
                    elif name[5] == 'T':  # Trauer -> Sadness
                        emotion.append('male_sad')
                        emov2.append(7)
                    elif name[5] == 'N':
                        emotion.append('male_neutral')
                        emov2.append(3)
                    else:
                        emotion.append('unknown')
                        emov2.append(0)
                        #print(name)
                    gender.append(0)
                else:
                    if name[5] == 'W':  # Ärger (Wut) -> Angry
                        emotion.append('female_angry')
                        emov2.append(1)
                    elif name[5] == 'L':  # Langeweile -> Boredom
                        emotion.append('female_bored')
                        emov2.append(2)
                    elif name[5] == 'E':  # Ekel -> Disgusted
                        emotion.append('female_disgust')
                        emov2.append(4)
                    elif name[5] == 'A':  # Angst -> Angry
                        emotion.append('female_fear')
                        emov2.append(5)
                    elif name[5] == 'F':  # Freude -> Happiness
                        emotion.append('female_happy')
                        emov2.append(6)
                    elif name[5] == 'T':  # Trauer -> Sadness
                        emotion.append('female_sad')
                        emov2.append(7)
                    elif name[5] == 'N':
                        emotion.append('female_neutral')
                        emov2.append(3)
                    else:
                        emotion.append('unknown')
                        emov2.append(0)
                    gender.append(1)
                #path.append(os.path.join( main_dir, name)) #"-", name)) #
                path.append( name)

        emodb_df = pd.DataFrame(emotion, columns=['labels'])
        emodb_emo1 = pd.DataFrame(emov2, columns=['emo'])
        emodb_df['source'] = 'EMODB'
        gender_df = pd.DataFrame(gender, columns=['gender'])
        emodb_df1 = pd.concat([emodb_df, emodb_emo1, gender_df, pd.DataFrame(path, columns=['path'])], axis=1)
        
        return emodb_df1

In [11]:
dfEmo=decompose_emodb()

In [12]:
dfEmo

,labels,source,emo,gender,path
0,male_happy,EMODB,6,0,03a01Fa.wav
1,male_neutral,EMODB,3,0,03a01Nc.wav
2,male_angry,EMODB,1,0,03a01Wa.wav
3,male_happy,EMODB,6,0,03a02Fc.wav
4,male_neutral,EMODB,3,0,03a02Nc.wav
...,...,...,...,...,...
530,female_bored,EMODB,2,1,16b10Lb.wav
531,female_sad,EMODB,7,1,16b10Tb.wav
532,female_sad,EMODB,7,1,16b10Td.wav
533,female_angry,EMODB,1,1,16b10Wa.wav


In [13]:
dfEmo[210:220]

,labels,source,emo,gender,path
210,male_fear,EMODB,5,0,11a07Ac.wav
211,male_bored,EMODB,2,0,11a07Ld.wav
212,male_sad,EMODB,7,0,11a07Ta.wav
213,male_angry,EMODB,1,0,11a07Wc.wav
214,male_fear,EMODB,5,0,11b01Ab.wav
215,male_disgust,EMODB,4,0,11b01Eb.wav
216,male_happy,EMODB,6,0,11b01Fc.wav
217,male_bored,EMODB,2,0,11b01Lb.wav
218,male_neutral,EMODB,3,0,11b01Nc.wav
219,male_angry,EMODB,1,0,11b01Wd.wav


In [14]:
dfEmo['path'][1]

'03a01Nc.wav'

In [15]:
dfEmo.loc[dfEmo['emo']==0]
#df.loc[df['col1'] == value]

,labels,source,emo,gender,path


In [16]:
X7=X6.copy()

In [17]:
X7

,0,1,2,3,4,5,6,7,8,9,...,183,184,185,186,187,188,189,190,191,192
0,-290.069550,120.892418,-44.022133,59.493820,-25.234184,24.166288,-20.102554,-3.104466,-3.912210,-8.433292,...,14.751010,17.490223,16.878034,56.398769,0.009679,0.012519,-0.031646,0.024728,0.007912,0.001966
1,-274.512543,137.867325,-33.016861,49.305836,-7.529971,15.471715,-6.811347,-7.069037,-8.804789,-1.528063,...,14.512919,17.550766,19.076122,53.806400,-0.001984,0.006133,-0.084608,0.109369,-0.025859,-0.003522
2,-225.866470,94.075874,-72.961555,41.311485,-27.577299,12.030844,-14.943604,-8.053857,-11.431667,-10.043902,...,16.131910,18.237524,19.299087,54.852081,0.001015,-0.006297,-0.033966,-0.076824,0.011315,-0.001692
3,-273.849060,98.794968,-36.564865,36.316208,-22.404598,18.368185,-12.455775,-1.077722,-12.487121,-17.107170,...,15.816742,17.617843,17.377833,58.313221,0.039030,-0.017753,-0.081100,-0.049775,0.015536,-0.007109
4,-252.887863,149.398911,-24.717367,45.297226,-0.930259,5.930630,0.473345,-0.681704,-12.864426,-9.714954,...,14.601713,14.795569,19.625183,51.336941,0.008184,-0.018455,-0.020061,-0.058433,0.009487,0.034003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,-255.980286,105.676674,-42.180950,75.896423,-23.686800,10.384299,-17.101816,-5.522917,-3.057154,-10.192598,...,16.838569,17.835524,19.693569,56.598065,0.013773,-0.044897,0.036112,-0.103853,0.014030,0.004264
531,-246.298492,91.399818,-25.200518,57.381245,-6.517654,19.359470,-4.424452,-3.416811,2.381373,-6.032307,...,15.286078,17.031323,18.757498,56.940429,0.022640,0.051658,0.111283,0.034628,0.032623,-0.023948
532,-287.566315,88.560669,-21.810999,60.376926,-0.650359,19.902145,-0.450192,2.266046,3.220632,-6.157897,...,14.402789,15.672307,17.590725,56.867307,0.064417,0.045792,-0.014448,0.036817,-0.002728,-0.039342
533,-329.608124,67.631531,-87.368507,64.361099,-49.395180,-1.466112,-20.686680,-10.990952,-6.811409,-28.455215,...,19.920409,19.619588,21.609127,53.589258,0.004312,0.004346,-0.046711,-0.008183,-0.011669,0.014701


In [18]:
X7 = pd.concat([X7,dfEmo['gender'],dfEmo['emo']],axis=1)

In [19]:
X7

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,gender,emo
0,-290.069550,120.892418,-44.022133,59.493820,-25.234184,24.166288,-20.102554,-3.104466,-3.912210,-8.433292,...,16.878034,56.398769,0.009679,0.012519,-0.031646,0.024728,0.007912,0.001966,0,6
1,-274.512543,137.867325,-33.016861,49.305836,-7.529971,15.471715,-6.811347,-7.069037,-8.804789,-1.528063,...,19.076122,53.806400,-0.001984,0.006133,-0.084608,0.109369,-0.025859,-0.003522,0,3
2,-225.866470,94.075874,-72.961555,41.311485,-27.577299,12.030844,-14.943604,-8.053857,-11.431667,-10.043902,...,19.299087,54.852081,0.001015,-0.006297,-0.033966,-0.076824,0.011315,-0.001692,0,1
3,-273.849060,98.794968,-36.564865,36.316208,-22.404598,18.368185,-12.455775,-1.077722,-12.487121,-17.107170,...,17.377833,58.313221,0.039030,-0.017753,-0.081100,-0.049775,0.015536,-0.007109,0,6
4,-252.887863,149.398911,-24.717367,45.297226,-0.930259,5.930630,0.473345,-0.681704,-12.864426,-9.714954,...,19.625183,51.336941,0.008184,-0.018455,-0.020061,-0.058433,0.009487,0.034003,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,-255.980286,105.676674,-42.180950,75.896423,-23.686800,10.384299,-17.101816,-5.522917,-3.057154,-10.192598,...,19.693569,56.598065,0.013773,-0.044897,0.036112,-0.103853,0.014030,0.004264,1,2
531,-246.298492,91.399818,-25.200518,57.381245,-6.517654,19.359470,-4.424452,-3.416811,2.381373,-6.032307,...,18.757498,56.940429,0.022640,0.051658,0.111283,0.034628,0.032623,-0.023948,1,7
532,-287.566315,88.560669,-21.810999,60.376926,-0.650359,19.902145,-0.450192,2.266046,3.220632,-6.157897,...,17.590725,56.867307,0.064417,0.045792,-0.014448,0.036817,-0.002728,-0.039342,1,7
533,-329.608124,67.631531,-87.368507,64.361099,-49.395180,-1.466112,-20.686680,-10.990952,-6.811409,-28.455215,...,21.609127,53.589258,0.004312,0.004346,-0.046711,-0.008183,-0.011669,0.014701,1,1


In [41]:
X7.to_csv("featureEmodb_Ek.csv",index=False,header=True)
#audio_df.to_csv("audioELM.csv",index=False,header=True)

In [19]:
dir ='D:/codes/datasets/ravdes/'
X7.to_excel(dir+"featureEmodbEk.xlsx",index=False) 

In [18]:
import os
data = pd.read_csv('featureShortc128.csv')
audio_df = pd.read_csv('audio_df.csv')

In [19]:
data

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,0.1,0.2
0,0.000002,0.000066,0.004286,0.024769,0.037116,0.020915,0.004085,0.022000,0.029166,0.024192,...,0.000002,7.983671e-07,4.238818e-07,4.620670e-07,3.084141e-07,7.357968e-08,1.610563e-08,1.311767e-09,1,1
1,0.000002,0.000093,0.004000,0.021978,0.060232,0.034005,0.003991,0.012792,0.034489,0.048722,...,0.000004,2.326391e-06,1.393828e-06,1.162513e-06,3.586766e-07,1.103367e-07,1.895200e-08,1.571617e-09,1,1
2,0.000004,0.000072,0.004056,0.025928,0.058986,0.018838,0.004335,0.025710,0.031794,0.059034,...,0.000007,5.198676e-06,4.122103e-06,2.653052e-06,1.764746e-06,3.578079e-07,5.847575e-08,6.194315e-09,1,1
3,0.000003,0.000084,0.007088,0.033968,0.076728,0.016350,0.003168,0.017293,0.028359,0.037432,...,0.000010,8.686617e-06,1.700473e-05,4.535204e-06,1.458934e-06,4.466640e-07,7.579326e-08,6.435894e-09,1,1
4,0.000003,0.000046,0.001499,0.013960,0.026140,0.028661,0.003578,0.009999,0.021098,0.016831,...,0.000002,3.709901e-06,1.328471e-06,3.481364e-07,1.555228e-07,5.434392e-08,1.003702e-08,8.769763e-10,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,0.000002,0.000019,0.000075,0.000058,0.000013,0.000023,0.000395,0.013317,0.032795,0.075172,...,0.000172,6.577667e-05,3.777974e-05,9.589142e-06,4.696875e-06,1.543086e-06,3.007084e-07,4.978052e-08,0,8
1436,0.000064,0.000131,0.000262,0.000223,0.000111,0.000136,0.002221,0.046049,0.076049,0.219650,...,0.000594,3.106455e-04,1.621839e-04,1.005938e-04,3.059142e-05,9.542340e-06,1.252834e-06,1.972792e-07,0,8
1437,0.000048,0.000030,0.000399,0.000541,0.000190,0.000160,0.000925,0.031382,0.103851,0.084082,...,0.000687,3.384231e-04,1.500613e-04,7.867455e-05,2.148737e-05,6.161907e-06,6.787151e-07,5.262246e-08,0,8
1438,0.003726,0.001217,0.000471,0.000263,0.000234,0.000296,0.001027,0.007759,0.036937,0.147467,...,0.003073,1.784341e-03,1.175709e-03,5.314132e-04,2.158249e-04,6.307891e-05,1.148216e-05,1.022560e-06,0,8


In [20]:
data.shape

(1440, 130)